In [1]:
import sys
from pandas import *
import numpy as np
import matplotlib
from matplotlib import pyplot
import random
from scipy.stats import norm
import os
import pybedtools

In [2]:
def read_JASPAR_transfac_pfms(infile):
    pfms_file = open(infile,'r')
    pfms_info = {}
    seq_len = 0
    for line in pfms_file:
        line = line.split('\n')[0]
        if(len(line.split(' ')) > 1):
            line = line.split(' ')
            if(line[0] == 'DE'):
                pfms_info['Matrix_ID'] = line[1]
                pfms_info['Matrix_Name'] = line[2]
                seq_len = 0
            elif(line[0] == 'CC'):
                temp = line[1].split(':')
                pfms_info[temp[0]] = temp[1]
                if(seq_len > 0):
                    pfms_info['TF_len'] = seq_len
        elif(len(line.split('\t')) > 1):
            line = line.split('\t')
            if(line[0] == 'PO'):
                curr_matorder = line[1:]
            else:
                curr_vals = {}
                for n,v in enumerate(line[1:]):
                    curr_vals[curr_matorder[n]] = float(v)+1
                pfms_info[int(line[0])] = curr_vals
                seq_len = int(line[0])
        else:
            pass
#             print(line)
    pfms_file.close()
    return pfms_info

In [3]:
def get_matrix_byTFName(tfname,info_dicts,seq): #old
    seq = seq.upper()
    matrix_dict_touse = None
    for i in info_dicts:
        if(i['Matrix_Name'] == tfname):
            matrix_dict_touse = i
            break
    if(matrix_dict_touse == None):
        print('Could not find a PWM for Transcription Factor {0}'.format(tfname))
        return None
    seq_val = 0
    for n,b in enumerate(seq):
        try:
            seq_val += float(matrix_dict_touse[n+1][b])
        except:
            if(b not in 'ACTG'):
                print('Sequence contains a letter, {0}, that is not A/C/G/T at position {1}'.format(b,n))
            else:
                print('Sequence of length {0} is too long'.format(len(seq)))
            return None
    return seq_val

In [4]:
def get_matrix_byTF(tfname,info_dicts):
    matrix_dict_touse = None
    for i in info_dicts:
        if(i['Matrix_Name'] == tfname):
            matrix_dict_touse = i
            break
    if(matrix_dict_touse == None):
        print('Could not find a PWM for Transcription Factor {0}'.format(tfname))
        return None
    return matrix_dict_touse

def get_lnPWM_from_matrixdict(matrix_dict):
    lnPWM_dict = {}
    for en in range(1,matrix_dict['TF_len']+1):
        temp_matrix = {}
        for b in 'ACTG':
            f = float(matrix_dict[en][b])
            if(f == 0.0):
                temp_matrix[b] = np.log(1)
            else:
                temp_matrix[b] = np.log(f)
        lnPWM_dict[en] = temp_matrix
    return lnPWM_dict

In [5]:
def get_fraclnPWM_from_matrixdict(matrix_dict):
    lnPWM_dict = {}
    for en in range(1,matrix_dict['TF_len']+1):
        temp_matrix = {}
        curr_totcount = sum([float(x) for x in matrix_dict[en].values()])
        for b in 'ACTG':
            f = float(matrix_dict[en][b])/curr_totcount
            if(f == 0.0):
                temp_matrix[b] = np.log(1)
            else:
                temp_matrix[b] = np.log(f)
        lnPWM_dict[en] = temp_matrix
    return lnPWM_dict

def get_fracPWM_from_matrixdict(matrix_dict):
    PWM_dict = {}
    for en in range(1,matrix_dict['TF_len']+1):
        temp_matrix = {}
        curr_totcount = sum([float(x) for x in matrix_dict[en].values()])
        for b in 'ACTG':
            if(f == 0.0):
                temp_matrix[b] = 1/curr_totcount
            else:
                temp_matrix[b] = float(matrix_dict[en][b])/curr_totcount
        PWM_dict[en] = temp_matrix
    return PWM_dict

In [6]:

def get_frac_matrix_scores(tfname,pwm,ref_al,alt_al,position,reference_seq):
    ref_al = ref_al.upper()
    alt_al = alt_al.upper()
    reference_seq = reference_seq.upper()
    ref_seqval_list,alt_seqval_list = [],[]
    for n,b in enumerate(reference_seq):
        try:
            if(n == position):
                ref_seqval_list.append(float(pwm[n+1][ref_al]))
                alt_seqval_list.append(float(pwm[n+1][alt_al]))
            else:
                ref_seqval_list.append(float(pwm[n+1][b]))
                alt_seqval_list.append(float(pwm[n+1][b]))
        except:
            if(b not in 'ACTG'):
                print('Sequence contains a letter, {0}, that is not A/C/G/T at position {1}'.format(b,n))
                return None
            else:
                continue

    return ref_seqval_list,alt_seqval_list


In [29]:
test_fPWM = get_fracPWM_from_matrixdict(test)
test_flnPWM = get_fraclnPWM_from_matrixdict(test)
test_pos_counts = get_matrix_counts(test,test['TF_len'])

In [3]:
chr(1)

'\x01'

In [62]:
# get_matrix_counts(test,test['TF_len'])
# test
get_matrix_score('DUX4',test_flnPWM,'ACCGTT')

[-1.6558171380122764,
 -10.551088053140093,
 -10.551114217794265,
 -2.9491337619027305,
 -0.8244308526280849,
 -0.7452422924433473]

In [7]:
def get_lnmatrix_scores(tfname,matrix_dict,ref_al,alt_al,position,reference_seq):
    ref_al = ref_al.upper()
    alt_al = alt_al.upper()
    reference_seq = reference_seq.upper()
    ref_seqval,alt_seqval = 0,0
    seqpos_counts = []
    seqpos_counts_bybase = {'A':[],'C':[],'G':[],'T':[],}
    for n,b in enumerate(reference_seq):
        try:
            if(n == position):
                ref_seqval += float(matrix_dict[n+1][ref_al])
                alt_seqval += float(matrix_dict[n+1][alt_al])
            else:
                ref_seqval += float(matrix_dict[n+1][b])
                alt_seqval += float(matrix_dict[n+1][b])
            curr_vals = [float(x) for x in matrix_dict[n+1].values()]
            seqpos_counts.append(sum(curr_vals))
            seqpos_counts_bybase['A'].append(matrix_dict[n+1]['A'])
            seqpos_counts_bybase['C'].append(matrix_dict[n+1]['C'])
            seqpos_counts_bybase['G'].append(matrix_dict[n+1]['G'])
            seqpos_counts_bybase['T'].append(matrix_dict[n+1]['T'])
        except:
            if(b not in 'ACTG'):
                print('Sequence contains a letter, {0}, that is not A/C/G/T at position {1}'.format(b,n))
                return None
            else:
                continue
        
#     total_count_fraction = seqpos_counts/len(seqpos_counts)
    refseq_fraction = ref_seqval/sum(seqpos_counts)
    altseq_fraction = alt_seqval/sum(seqpos_counts)
    return ref_seqval,alt_seqval,seqpos_counts_bybase

In [8]:
def get_numbp_forseq(seq):
    bpcounts = {'A':0,'C':0,'G':0,'T':0}
    for b in seq:
        try:
            bpcounts[b] += 1
        except:
            if(b not in 'ACTG'):
                print('Sequence contains a letter, {0}, that is not A/C/G/T!'.format(b))
    bpc_list = []
    for o in 'ACGT':
        bpc_list.append(bpcounts[o])
    return bpc_list
            
def compute_Y(seq,bg_props):
#     seq_bpcounts = get_numbp_forseq(seq)
    bpcounts = {'A':0,'C':0,'G':0,'T':0}
    for b in seq:
        try:
            bpcounts[b] += 1
        except:
            if(b not in 'ACTG'):
                print('Sequence contains a letter, {0}, that is not A/C/G/T!'.format(b))
    bpc_list = []
    for o in 'ACGT':
        bpc_list.append(bpcounts[o])
    Y = np.dot(np.log(bg_props),bpc_list)
    return Y

def make_seq(fullseq,position,allele):
    new_seq = ''
    for n,b in enumerate(fullseq):
        try:
            if(n==position):
                new_seq = ''.join([new_seq,allele])
            else:
                new_seq = ''.join([new_seq,b])
        except:
            if(b not in 'ACTG'):
                print('Sequence contains a letter, {0}, that is not A/C/G/T at position {1}'.format(b,n))
                return None
            else:
                print(b)
                continue
    return new_seq.upper()

def get_matrix_counts(pwm,seqlen):
    pos_counts = []
    for n in range(1,seqlen+1):
        temp = [float(x) for x in pwm[n].values()]
        pos_counts.append(sum(temp))
    return pos_counts

In [9]:

def get_lnPWM_from_fracPWM(fracPWM,bgfreqs):
    lnPWM_dict = {}
    for en in range(1,len(fracPWM)+1):
        temp_matrix = {}
        for b in 'ACTG':
            f = float(fracPWM[en][b])/bgfreqs['frac_{0}'.format(b)].values[0]
            if(f == 0.0):
                print('fraction is 0')
                temp_matrix[b] = np.log(1)
            else:
                temp_matrix[b] = -np.log(f)
        lnPWM_dict[en] = temp_matrix
    return lnPWM_dict

def get_fracPWM_from_matrixdict(matrix_dict):
    PWM_dict = {}
    for en in range(1,matrix_dict['TF_len']+1):
        temp_matrix = {}
        curr_totcount = sum([float(x) for x in matrix_dict[en].values()])
        for b in 'ACTG':
            if(f == 0.0):
                temp_matrix[b] = 1/curr_totcount
            else:
                temp_matrix[b] = float(matrix_dict[en][b])/curr_totcount
        PWM_dict[en] = temp_matrix
    return PWM_dict

In [10]:
def get_matrix_scores(pwm,seq):
    seqval_list = []
    for n,b in enumerate(seq):
        try:
            seqval_list.append(float(pwm[n+1][b]))
        except:
            if(b not in 'ACTG'):
                print('Sequence contains a letter, {0}, that is not A/C/G/T at position {1}'.format(b,n))
                return None
            else:
                continue
    return seqval_list
def get_complseq(seq):
    new_seq = []
    for b in seq:
        if(b == 'A'):
            new_seq.append('T')
        elif(b == 'T'):
            new_seq.append('A')
        elif(b == 'G'):
            new_seq.append('C')
        elif(b == 'C'):
            new_seq.append('G')
        else:
            print('Base pair not A/C/T/G! {0}'.format(b))
    return ''.join(new_seq)
            

In [11]:
test = get_matrix_byTF('DUX4',infodicts_list)
ln_test = get_lnPWM_from_matrixdict(test)
test_counts = get_matrix_counts(test,11)
test_matscores = get_matrix_scores('DUX4',test,'ACCTGGACATG')

NameError: name 'infodicts_list' is not defined

In [62]:
# test
# r,a,c = get_lnmatrix_scores('DUX4',ln_test,'C','A',2,'ACCTG')
test_ln_fracPWM = get_fracPWM_from_matrixdict(test)
ln_test = get_lnPWM_from_matrixdict(test)

In [65]:
get_lnPWM_from_fracPWM(test_ln_fracPWM,test_bgfreqs)

{1: {'A': 0.4344962438862845,
  'C': 1.1435191821247397,
  'T': -0.8888460955971449,
  'G': 2.0788455465242905},
 2: {'A': -1.1888753833368704,
  'C': 8.965963911614057,
  'T': 9.3311660557505,
  'G': 1.8622540252013808},
 3: {'A': -1.2212100269186392,
  'C': 8.965963911614057,
  'T': 9.3311660557505,
  'G': 8.966398118188907},
 4: {'A': 0.7678765266945468,
  'C': -0.39932682035994754,
  'T': -0.5373676675220993,
  'G': 1.3639967825230899},
 5: {'A': 9.329851860882677,
  'C': -0.7438782103966722,
  'T': -0.3955246506343733,
  'G': 0.45281142236678246},
 6: {'A': 9.329851860882677,
  'C': -0.5774142345347029,
  'T': -0.47470866791862754,
  'G': 0.24577440676848086},
 7: {'A': -1.1411913041651307,
  'C': 0.9969521305075801,
  'T': 5.399340423026175,
  'G': 8.966398118188907},
 8: {'A': -1.2212100269186392,
  'C': 8.965963911614057,
  'T': 9.3311660557505,
  'G': 8.966398118188907},
 9: {'A': 9.329851860882677,
  'C': 8.965963911614057,
  'T': -1.2198958320508166,
  'G': 8.966398118188907

In [40]:
test_seq = 'ACCTGG'
test_bgfreqs1 = np.array(bgfrac_df.loc[bgfrac_df['Chrm'] == 'Total'][['frac_A','frac_C','frac_G','frac_T']])

In [28]:
test_bgfreqs = bgfrac_df.loc[bgfrac_df['Chrm'] == 'Total'][['frac_A','frac_C','frac_G','frac_T']]

In [42]:
test_bgfreqs1['A']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [12]:
gene_df = read_csv('/local3/jake/TF_binding/14.5M.TopVars.LE.chrm8.pos61768012.TF_genes',header=None,delimiter='\t')
gene_df.columns = ['pos_start','pos_end','tf_name']
position = 61768012
chromosome = 8
ref_pos_end = max(gene_df['pos_end'])
ref_pos_start = min(gene_df['pos_start'])
gene_df['relative_start'] =  gene_df['pos_start']-ref_pos_start
gene_df['relative_end'] = gene_df['pos_end']-ref_pos_start

matrix_loc = '/local3/jake/TF_binding/JASPAR2020_CORE_vertebrates_non-redundant_pfms_transfac/'
# matrix_loc = 'gene_dfJASPAR2020_CORE_vertebrates_non-redundant_pfms_transfac'
transfac_matrix_list = os.listdir(matrix_loc)
infodicts_list = []
for f in transfac_matrix_list:
    curr_infodict = read_JASPAR_transfac_pfms('{0}/{1}'.format(matrix_loc,f))
    infodicts_list.append(curr_infodict)
    
# for i,g in gene_df.iterrows():
#     curr_score = get_matrix_byTFName(g[2],infodicts_list,'ACT')
#     print('score for {0} is {1}'.format(g[2],curr_score))
    

# ref_full_seq = get_hg19reference_sequence(ref_pos_start,ref_pos_end,'/local3/jake/TF_binding/chr4.fa')
ref_full_seq = pybedtools.BedTool.seq('chr{0}:{1}-{2}'.format(8,ref_pos_start,(ref_pos_end)),'/local3/jake/TF_binding/chr8.fa')

updated_pos = (position-ref_pos_start)

FileNotFoundError: [Errno 2] File b'/local3/jake/TF_binding/14.5M.TopVars.LE.chrm8.pos61768012.TF_genes' does not exist: b'/local3/jake/TF_binding/14.5M.TopVars.LE.chrm8.pos61768012.TF_genes'

In [11]:
bgfrac_df = read_csv('/local3/jake/TF_binding/ACTG_count.all_chrms.fractions.v2.txt',delimiter='\t')
score_dict_bytf ={}
for i,g in gene_df.iterrows():
    curr_relative_pos = abs(updated_pos-g['relative_start'])
    curr_refseq = make_seq(ref_full_seq[g['relative_start']:(g['relative_end']+1)],curr_relative_pos,'G')
    curr_altseq = make_seq(ref_full_seq[g['relative_start']:(g['relative_end']+1)],curr_relative_pos,'A')
    curr_matrix = get_matrix_byTF(g['tf_name'],infodicts_list)
    curr_fracPWM = get_fracPWM_from_matrixdict(curr_matrix)
    curr_lnfracPWM = get_fraclnPWM_from_matrixdict(curr_matrix)
    # curr_refscore,curr_altscore,curr_counts = get_matrix_scores(tfname=g['tf_name'],matrix_dict=curr_matrix,ref_al=args.ref_al,alt_al=args.alt_al,position=curr_relative_pos,reference_seq=curr_seq)
    # curr_totcount = sum(curr_counts)
    refscore_list = get_matrix_scores(curr_matrix,curr_refseq)
    altscore_list = get_matrix_scores(curr_matrix,curr_altseq)
    pos_counts = get_matrix_counts(curr_matrix,curr_matrix['TF_len'])
    tot_count = sum(pos_counts)
    curr_scoredict = {'ref_score':sum(refscore_list),'alt_score':sum(altscore_list),'tf_len':curr_matrix['TF_len'],'counts_perpos':min(pos_counts)}
    curr_scoredict['ref_fraction_score'] = sum([refscore_list[x]/pos_counts[x] for x in range(len(pos_counts))])
    curr_scoredict['alt_fraction_score'] = sum([altscore_list[x]/pos_counts[x] for x in range(len(pos_counts))])

    refscore_ln = get_matrix_scores(curr_lnfracPWM,curr_refseq)
    altscore_ln = get_matrix_scores(curr_lnfracPWM,curr_altseq)
    curr_scoredict['ref_fraction_ln_score'] = sum(refscore_ln)
    curr_scoredict['alt_fraction_ln_score'] = sum(altscore_ln)

    # frac_refvals,frac_altvals = get_frac_matrix_scores(tfname=g['tf_name'],pwm=curr_fracPWM,ref_al=args.ref_al,alt_al=args.alt_al,position=curr_relative_pos,reference_seq=curr_seq)
    # fracln_refvals,fracln_altvals = get_frac_matrix_scores(tfname=g['tf_name'],pwm=curr_lnfracPWM,ref_al=args.ref_al,alt_al=args.alt_al,position=curr_relative_pos,reference_seq=curr_seq)
    bgfreqs = np.array(bgfrac_df.loc[bgfrac_df['Chrm'] == str(chromosome)][['frac_A','frac_C','frac_G','frac_T']])
    ref_Y = compute_Y(curr_refseq,bgfreqs)
    alt_Y = compute_Y(curr_altseq,bgfreqs)
    ref_H = refscore_ln - ref_Y
    alt_H = altscore_ln - alt_Y
    curr_scoredict['H'] = np.sum(ref_H)
    curr_scoredict['Hprime'] = np.sum(alt_H)

    score_dict_bytf[g['tf_name']] = curr_scoredict
    

In [66]:
seqA = 'AAAAAAAAAAAAAA'
seqB = 'GAAAAAAAAAAAAA'
curr_matrix = read_JASPAR_transfac_pfms('/local3/jake/TF_binding/TF_1.transfac')
bgfreqs = bgfrac_df.loc[bgfrac_df['Chrm'] == 'Total'][['frac_A','frac_C','frac_G','frac_T']]
# curr_lnfracPWM = get_fraclnPWM_from_matrixdict(curr_matrix)
curr_fracPWM = get_fracPWM_from_matrixdict(curr_matrix)
curr_lnfracPWM = get_lnPWM_from_fracPWM(curr_fracPWM,bgfreqs)
Ascore_ln = get_matrix_scores(curr_lnfracPWM,seqA)
Bscore_ln = get_matrix_scores(curr_lnfracPWM,seqB)

A_Y = compute_Y(seqA,np.array(bgfreqs))
B_Y = compute_Y(seqB,np.array(bgfreqs))
A_H = Ascore_ln - A_Y
B_H = Bscore_ln - B_Y

In [67]:
np.sum(Ascore_ln)

-8.983879845730163

In [68]:
np.sum(Bscore_ln)

-7.158948847656853

In [13]:
def compute_Y(seq,bg_props):
#     seq_bpcounts = get_numbp_forseq(seq)
    bpcounts = {'A':0,'C':0,'G':0,'T':0}
    for b in seq:
        try:
            bpcounts[b] += 1
        except:
            if(b not in 'ACTG'):
                print('Sequence contains a letter, {0}, that is not A/C/G/T!'.format(b))
    bpc_list = []
    for o in 'ACGT':
        bpc_list.append(bpcounts[o])
    Y = np.dot(np.log(bg_props),bpc_list)
    return Y

In [162]:
# sum_A = np.sum(bgfrac_df['count_A'])
# sum_C = np.sum(bgfrac_df['count_C'])
# sum_G = np.sum(bgfrac_df['count_G'])
# sum_T = np.sum(bgfrac_df['count_T'])
# sum_total = np.sum(bgfrac_df['total'])
# bgfrac_df = bgfrac_df.append({'Chrm':'Total','count_A':sum_A,'count_C':sum_C,'count_G':sum_G,'count_T':sum_T,'total':sum_total,'frac_A':(sum_A/sum_total),'frac_C':(sum_C/sum_total),'frac_G':(sum_G/sum_total),'frac_T':(sum_T/sum_total)},ignore_index=True)
np.array(bgfrac_df.loc[bgfrac_df['Chrm'] == 8][['frac_A','frac_C','frac_G','frac_T']])


array([], shape=(0, 4), dtype=float64)

In [70]:
get_complseq('AAAC')

'TTTG'

In [153]:
bgfrac_df.to_csv('/local3/jake/TF_binding/ACTG_count.all_chrms.fractions.v2.txt',sep='\t',index=None)

In [13]:
results_loc = '/local3/jake/TF_binding/tophit_SNPs/output_v2/'
pwmresults_list = os.listdir(results_loc)

assocfile_loc = '/local3/mhansen/Projects/CTA/Analysis/GWAS/All_Inds/Results/5M_Filtered'
infofile_loc = '/local3/jake/TF_binding/tophit_SNPs/'
infofile_list = os.listdir(infofile_loc)

final_df_dictlist = []
for i in infofile_list:
    if('.full_assoc' not in i):
        continue
    curr_df = read_csv('{0}{1}'.format(infofile_loc,i),delimiter='\t')
    curr_prefix = '.'.join(i.split('.')[:-1])
    temp_dict = {}
    for num,snp in curr_df.iterrows():
        temp_dict['chr'] = snp['#Chr']
        temp_dict['pos'] = snp['Pos']
        temp_dict['snp_id'] = snp['VID']
        temp_dict['beta'] = snp['Beta']
        temp_dict['pvalue'] = snp['pval']
        temp_dict['ref_allele'] = snp['Ref']
        temp_dict['alt_allele'] = snp['Alt']
        temp_dict['trait_num'] = curr_prefix.split('.')[0]
        curr_pwmfile = '{0}{1}.chrm{2}.pos{3}.PWM_scores'.format(results_loc,curr_prefix,snp['#Chr'],snp['Pos'])
        

KeyError: 'Beta'

In [17]:
test_pwmfile = '24.5M.TopVars.LE.chrm4.pos165034923.PWM_scores'
# position = int(args.position.split(':')[1])
position = 165034923
# chromosome = int(args.position.split(':')[0])
chromosome = 4

# gene_df = read_csv('{0}'.format(args.input_gene_file),header=None,delimiter='\t')
gene_df = read_csv('/local3/jake/TF_binding/tophit_SNPs/output/24.5M.TopVars.LE.chrm4.pos165034923.TF_genes',header=None,delimiter='\t')
gene_df.columns = ['pos_start','pos_end','tf_name']
# transfac_matrix_list = os.listdir(args.matrix_loc)
matrix_loc = '/local3/jake/TF_binding/JASPAR2020_CORE_vertebrates_non-redundant_pfms_transfac/'
# matrix_loc = 'gene_dfJASPAR2020_CORE_vertebrates_non-redundant_pfms_transfac'
transfac_matrix_list = os.listdir(matrix_loc)
infodicts_list = []
for f in transfac_matrix_list:
    curr_infodict = read_JASPAR_transfac_pfms('{0}/{1}'.format(matrix_loc,f))
    infodicts_list.append(curr_infodict)

ref_pos_end = max(gene_df['pos_end'])
ref_pos_start = min(gene_df['pos_start'])
# ref_full_seq = get_hg19reference_sequence(ref_pos_start,ref_pos_end,args.ref_fasta_file)
ref_full_seq = pybedtools.BedTool.seq('chr{0}:{1}-{2}'.format(chromosome,ref_pos_start,ref_pos_end),args.ref_fasta_file)
updated_pos = (position-ref_pos_start)
gene_df['relative_start'] =  gene_df['pos_start']-ref_pos_start
gene_df['relative_end'] = gene_df['pos_end']-ref_pos_start

score_dict_bytf ={}
for i,g in gene_df.iterrows():
    curr_relative_pos = abs(updated_pos-g['relative_start'])
    curr_seq = ref_full_seq[g['relative_start']:(g['relative_end']+1)]
    curr_matrix = get_matrix_byTF(g['tf_name'],infodicts_list)
    curr_refscore,curr_altscore,curr_counts = get_matrix_scores(tfname=g['tf_name'],matrix_dict=curr_matrix,ref_al=args.ref_al,alt_al=args.alt_al,position=curr_relative_pos,reference_seq=curr_seq)
    curr_totcount = sum(curr_counts)
    score_dict_bytf[g['tf_name']] = {'ref_score':curr_refscore,'alt_score':curr_altscore,'ref_fraction_score':(curr_refscore/curr_totcount),'alt_fraction_score':(curr_altscore/curr_totcount),'tf_len':len(curr_seq),'counts_perpos':curr_counts[0]}


NameError: name 'args' is not defined

In [14]:
results_loc = '/local3/jake/TF_binding/tophit_SNPs/output_v3/'
pwmresults_list = os.listdir(results_loc)

assocfile_loc = '/local3/mhansen/Projects/CTA/Analysis/GWAS/All_Inds/Results/5M_Filtered'
infofile_loc = '/local3/jake/TF_binding/tophit_SNPs/'
infofile_list = os.listdir(infofile_loc)

final_df_dictlist = []
for i in infofile_list:
    if('.full_assoc' not in i):
        continue
    elif(i == '8.1M_5M.TopVars.LE.5M.full_assoc' or i == '7.1M_5M.TopVars.LE.5M.full_assoc'):
        continue
    curr_df = read_csv('{0}{1}'.format(infofile_loc,i),delimiter='\t')
    curr_prefix = '.'.join(i.split('.')[:-1])
    temp_dict = {}
    for num,snp in curr_df.iterrows():
        temp_dict['chr'] = snp['#Chr']
        temp_dict['pos'] = snp['Pos']
        temp_dict['snp_id'] = snp['VID']
        if(i == '8.1M_5M.TopVars.LE.full_assoc' or i == '7.1M_5M.TopVars.LE.full_assoc' ):
            temp_dict['beta'] = snp['Beta_FE']
            temp_dict['pvalue'] = snp['pval_FE']
            if(i == '8.1M_5M.TopVars.LE.full_assoc'):
                d = read_csv('{0}{1}'.format(infofile_loc,'8.1M_5M.TopVars.LE.5M.full_assoc'),delimiter='\t')
                temp_dict['ref_allele'] = d.loc[num]['Ref']
                temp_dict['alt_allele'] = d.loc[num]['Alt']
            if(i == '7.1M_5M.TopVars.LE.full_assoc'):
                d = read_csv('{0}{1}'.format(infofile_loc,'7.1M_5M.TopVars.LE.5M.full_assoc'),delimiter='\t')
                temp_dict['ref_allele'] = d.loc[num]['Ref']
                temp_dict['alt_allele'] = d.loc[num]['Alt']
        else:
            temp_dict['pvalue'] = snp['pval']
            temp_dict['beta'] = snp['Beta']
            temp_dict['ref_allele'] = snp['Ref']
            temp_dict['alt_allele'] = snp['Alt']
        temp_dict['trait_num'] = curr_prefix.split('.')[0]
        curr_pwmfile = '{0}{1}.chrm{2}.pos{3}.PWM_scores'.format(results_loc,curr_prefix,snp['#Chr'],snp['Pos'])
        curr_pwm_df = read_csv(curr_pwmfile,skiprows=1,delimiter='\t')
        for index,tf in curr_pwm_df.iterrows():
            curr_dict = temp_dict.copy()
            curr_dict['TF_name'] = tf['TF_Name']
            curr_dict['PWM_fracscore_ref'] = "{0:.5f}".format(tf['PWM Fraction Score (REF allele)'])
            curr_dict['PWM_fracscore_alt'] = "{0:.5f}".format(tf['PWM Fraction Score (ALT allele)'])
            curr_dict['TF_len'] = tf['TF Length']
            curr_dict['TF_count'] = tf['TF Counts per position']
            curr_dict['PWM_ln_fracscore_ref'] = "{0:.5f}".format(tf['REF Log(PWM Fraction Score)'])
            curr_dict['PWM_ln_fracscore_alt'] = "{0:.5f}".format(tf['ALT Log(PWM Fraction Score)'])
            curr_dict['H (REF)'] = "{0:.5f}".format(tf['H (REF)'])
            curr_dict['Hprime (ALT)'] = "{0:.5f}".format(tf['Hprime (ALT)'])
            final_df_dictlist.append(curr_dict)

final_df = DataFrame(final_df_dictlist)



In [23]:
for i in infofile_list:
    if('.full_assoc' not in i):
        continue
    elif(i == '8.1M_5M.TopVars.LE.5M.full_assoc' or i == '7.1M_5M.TopVars.LE.5M.full_assoc'):
        continue
    if(i == '8.1M_5M.TopVars.LE.full_assoc' or i == '7.1M_5M.TopVars.LE.full_assoc' ):
        print(i)
        


7.1M_5M.TopVars.LE.full_assoc
8.1M_5M.TopVars.LE.full_assoc


In [29]:
final_df.to_csv('/local3/jake/TF_binding/tophit_SNPs/final_PWM.alltraits.allsnps.output_v3',sep='\t',index=False)

In [15]:
final_df

,chr,pos,snp_id,pvalue,beta,ref_allele,alt_allele,trait_num,TF_name,PWM_fracscore_ref,PWM_fracscore_alt,TF_len,TF_count,PWM_ln_fracscore_ref,PWM_ln_fracscore_alt,H (REF),Hprime (ALT)
0,3,13308152,3:13308152,1.252538e-11,6.312838,C,A,6,TFAP2A(var.2),4.92927,5.30035,12,4009.0,-18.36669,-16.88040,213.84023,210.31008
1,3,13308152,3:13308152,1.252538e-11,6.312838,C,A,6,BARX2,3.84562,4.58265,12,28391.0,-32.67471,-26.33297,194.51138,195.83667
2,3,13308152,3:13308152,1.252538e-11,6.312838,C,A,6,PAX5,4.02331,4.07698,12,1416.0,-25.29941,-24.16443,196.86585,192.98438
3,3,13308152,3:13308152,1.252538e-11,6.312838,C,A,6,ESRRA,3.95458,3.75291,13,25999.0,-29.14707,-30.05458,227.12275,220.78075
4,3,13308152,3:13308152,1.252538e-11,6.312838,C,A,6,RHOXF1,3.59007,3.56901,8,10920.0,-22.44783,-22.51674,83.95005,80.53684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4213,6,4907848,6:4907848,2.908110e-08,2.545850,A,G,8,ZBTB6,1.48991,1.48991,13,2231.0,-38.22970,-38.22970,218.08801,223.56885
4214,6,4907848,6:4907848,2.908110e-08,2.545850,A,G,8,Dux,1.46315,1.46315,8,1003.0,-37.49455,-37.49455,58.84941,62.22224
4215,6,4907848,6:4907848,2.908110e-08,2.545850,A,G,8,DUX4,1.45930,1.45930,11,38221.0,-65.74039,-65.74039,115.53229,120.16992
4216,6,4907848,6:4907848,2.908110e-08,2.545850,A,G,8,Sox5,1.44444,1.29630,7,27.0,-16.35184,-16.93963,59.56230,61.92574


In [36]:
genomesize_file = open('/local3/jake/TF_binding/genomesize.all_chrms.hg19.txt','w')
for row,c in bgfrac_df.iterrows():
    genomesize_file.write('chr{0}\t{1}\n'.format(int(c['Chrm']),int(c['total'])))

genomesize_file.close()

In [28]:
ref_full_seq = pybedtools.BedTool.seq('chr{0}:{1}-{2}'.format(chromosome,ref_pos_start,ref_pos_end),'/local3/jake/TF_binding/chr8.fa')
tfname_list = list(set(final_df['TF_name']))


In [51]:
# pybedtools.BedTool.random(genome='/local3/jake/TF_binding/genomesize.all_chrms.hg19.txt',l=10, n=10)


TypeError: decorated() missing 1 required positional argument: 'self'

In [23]:
def get_random_bgseqs(slen,reps,fastaloc):
    x = pybedtools.BedTool()
    
    bgseq_list = []
    nreps = 0
    while(len(bgseq_list) < reps):
        nreps += 1
        y = x.random(l=slen, n=1, genome='hg19')
        for s in y:
            try:
#                 curr_chrm = int(s[0].split('r')[1])
                curr_chrm = 8
                curr_start = int(s[1])
                curr_end = int(s[2])
                chrmfile = '{0}/chr{1}.fa'.format(fastaloc,curr_chrm)
                bgseq_list.append({'chrm':curr_chrm,'start':curr_start,'end':curr_end,'seq':'AAAC'})
            except:
                continue
#         try:
#             curr_seq = pybedtools.BedTool.seq('chr{0}:{1}-{2}'.format(curr_chrm,curr_start,curr_end),chrmfile)
#             bgseq_list.append({'chrm':curr_chrm,'start':curr_start,'end':curr_end,'seq':curr_seq.upper()})
#         except:
#             print('Could not find fasta file! Check the location and try again.')
#             return -1
        
    return bgseq_list

def get_random_bgseqs_v2(slen,reps,chrm,bgfrac_df,chrmfile):
    csize = bgfrac_df.loc[bgfrac_df['Chrm'] == chrm]['total'].values[0]
    bgseq_list = []
    for s in range(reps):
        curr_start = random.randint(0,(csize-slen))
        curr_seq = pybedtools.BedTool.seq('chr{0}:{1}-{2}'.format(chrm,curr_start,(curr_start+slen)),chrmfile)
        bgseq_list.append({'start':curr_start,'end':(curr_start+slen),'seq':curr_seq.upper()}) 
    return bgseq_list

In [16]:
get_random_bgseqs(10,5,'/local3/jake/TF_binding')

[{'chrm': 8, 'start': 158615095, 'end': 158615105, 'seq': 'TEST'},
 {'chrm': 8, 'start': 146653718, 'end': 146653728, 'seq': 'TEST'},
 {'chrm': 8, 'start': 128834204, 'end': 128834214, 'seq': 'TEST'},
 {'chrm': 8, 'start': 111433622, 'end': 111433632, 'seq': 'TEST'},
 {'chrm': 8, 'start': 44903868, 'end': 44903878, 'seq': 'TEST'}]

In [73]:
def calculate_bgH(seq,ln_pwm,bgfreqs):
    currscore_ln = get_matrix_scores(ln_pwm,seq)
    Y = compute_Y(seq,bgfreqs)
    H = currscore_ln - Y
    return np.sum(H)

In [27]:
bgseqs = get_random_bgseqs(10,5,'/local3/jake/TF_binding')
bg_H_list = []
bg_H_by_TF = {}
for s in bgseqs:
    curr_seq = s['seq']
    curr_matrix = get_matrix_byTF('TEF',infodicts_list)
    bgfreqs = bgfrac_df.loc[bgfrac_df['Chrm'] == '8'][['frac_A','frac_C','frac_G','frac_T']]
    curr_fracPWM = get_fracPWM_from_matrixdict(curr_matrix)
    curr_lnfracPWM = get_lnPWM_from_fracPWM(curr_fracPWM,bgfreqs)
    curr_H = np.sum(get_matrix_scores(curr_lnfracPWM,curr_seq))
    bg_H_list.append(curr_H)
#     bg_H_list.append()
bg_H_by_TF['TEST_TF'] = bg_H_list

In [31]:
for tf,h in bg_H_by_TF.items():
    print(tf,np.sum(h))

TEST_TF 46.56055507754986


In [111]:
for i,g in gene_df.iterrows():
    curr_relative_pos = abs(updated_pos-g['relative_start'])
    curr_seq = ref_full_seq[g['relative_start']:(g['relative_end']+1)]
    
    curr_matrix = get_matrix_byTF(g['tf_name'],infodicts_list)
#     print(curr_matrix,curr_seq)
    curr_refscore,curr_altscore = get_matrix_scores(tfname=g['tf_name'],matrix_dict=curr_matrix,ref_al='T',alt_al='G',position=curr_relative_pos,reference_seq=curr_seq)
#     print(g['tf_name'],curr_refscore,curr_altscore)
#     print((g['pos_start']-ref_pos_start),(g['pos_end']-ref_pos_start),""(position-ref_pos_start))
#     print(g['pos_start'],g['pos_end'],position)
#     print(g['relative_start'],g['relative_end'],curr_relative_pos,updated_pos)
    print(curr_seq,curr_seq[curr_relative_pos])

ValueError: too many values to unpack (expected 2)

In [201]:
for tf,scores in sorted(score_dict_bytf.items(), key=lambda k_v: k_v[1]['alt_score'],reverse=True):
    print(tf,scores)

OTX2 {'ref_score': 264488.0, 'alt_score': 148783.0}
KLF5 {'ref_score': 72298.0, 'alt_score': 72429.0}
NR2C2(var.2) {'ref_score': 28556.0, 'alt_score': 47184.0}
PITX1 {'ref_score': 41886.0, 'alt_score': 31355.0}
RHOXF1 {'ref_score': 40212.0, 'alt_score': 29296.0}
NR2C1 {'ref_score': 16249.0, 'alt_score': 25871.0}
PITX3 {'ref_score': 27003.0, 'alt_score': 20765.0}
THRB {'ref_score': 14938.0, 'alt_score': 15644.0}
HOXB6 {'ref_score': 12451.0, 'alt_score': 13489.0}
GSC2 {'ref_score': 8967.0, 'alt_score': 6951.0}
RARA::RXRG {'ref_score': 3092.0, 'alt_score': 4033.0}
VDR {'ref_score': 1707.0, 'alt_score': 1538.0}
SOX18 {'ref_score': 1374.0, 'alt_score': 925.0}
Dmbx1 {'ref_score': 396.0, 'alt_score': 405.0}
MZF1(var.2) {'ref_score': 17.0, 'alt_score': 28.0}


In [39]:
results_loc = '/local3/jake/TF_binding/tophit_SNPs/output/'
results_files_list = os.listdir(results_loc)
results_files_bytraitcode = {}
for f in results_files_list:
    if(f.split('.')[-1] != 'PWM_scores'):
        continue
    traitcode = int(f.split('.')[0]) #".".join(f.split('.')[:2])
    if(traitcode not in results_files_bytraitcode.keys()):
        results_files_bytraitcode[traitcode] = []
    results_files_bytraitcode[traitcode].append(f)

In [34]:
bgfrac_df

,Chrm,count_A,count_C,count_G,count_T,total,frac_A,frac_C,frac_G,frac_T
0,1,65570891,47024412,47016562,65668756,225280621.0,0.291063,0.208737,0.208702,0.291498
1,2,71102632,47915465,47947042,71239379,238204518.0,0.298494,0.201153,0.201285,0.299068
2,3,58713343,38653197,38670110,58760485,194797135.0,0.301408,0.198428,0.198515,0.301650
3,4,57932980,35885806,35890822,57952068,187661676.0,0.308710,0.191226,0.191253,0.308811
4,5,53672554,35089383,35129186,53804137,177695260.0,0.302048,0.197469,0.197693,0.302789
5,6,50554433,33143287,33163423,50533923,167395066.0,0.302007,0.197994,0.198115,0.301884
6,7,45997757,31671670,31636979,46047257,155353663.0,0.296084,0.203868,0.203645,0.296403
7,8,42767293,28703983,28702621,42715025,142888922.0,0.299304,0.200883,0.200874,0.298939
8,9,35260078,24826212,24813259,35243882,120143431.0,0.293483,0.206638,0.206530,0.293348
9,10,38330752,27308648,27298423,38376915,131314738.0,0.291900,0.207963,0.207885,0.292251
